# Part III: 
# Deep Learning Methods
Jason Brownlee

So, i commited a terrible mistake by letting me think that by just copying some code from the web and doing multistep forecasting we were actually assessing the performance of deep learning methods on chaotic time series data. Oh boy, i was wrong. So to fix it which is done by learning what the fuck is going on, cause i have really bad performance with my naive LSTM using my rossler data for the 5x20_sample.csv

## Chapter 6:
## How to Prepare Time Series Data for CNNs and LSTMs

### 6.1 Overview

THis tutorial is divided into three parts, they are:
1. Time Series to Supervised
2. 3D Data Preparation Basics
3. Univariate Worked Example

### 6.2 Time Series to Supervised

In [ ]:
# transform univariate time series to supervised learning problem
from numpy import array

# split a univariate sequence into samples
def split_sequence(sequence, n_steps):
    X, y = list(), list()
    for i in range(len(sequence)):
      # find the end of this pattern
      end_ix = i + n_steps
      # check if we are beyond the sequence
      if end_ix > len(sequence)-1:
         break
      # gather input and outpu parts of the pattern
      seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
      X.append(seq_x)
      y.append(seq_y)

    return array(X), array(y)

# define univariate time series
series = array([i for i in range(1, 11)])
print(series.shape)
# transform to a supervised learning problem
X, y = split_sequence(series, 3)
print(X.shape, y.shape)
# show each example
for i in range(len(X)):
   print(X[i], y[i])

(10,)
(7, 3) (7,)
[1 2 3] 4
[2 3 4] 5
[3 4 5] 6
[4 5 6] 7
[5 6 7] 8
[6 7 8] 9
[7 8 9] 10


### 6.3 3D Data Preparation Basics

Preparing time series data for CNNs and LSTMs requires one additional step beyond transforming the data into a supervised learning problem.

The input layer for CNN and LSTM modes is specified by the `input_shape` argument on the first hidden layer of the network. This is confuding cause we intuitively may expect the first layer defined in the model be the input layer, not the first hidden layer.

For example, below is an example of a network with one hidden `LSTM` layer and one `Dense` output layer.

```python
# lstm without an input layer
...
model = Sequential()
model.add(LSTM(32))
model.add(Dense(1))

In this example, the `LSTM()` layer must specify the shape of the input data. The input to every CNN and LSTM layer must be three-dimensional. The three dimensions of this input are:

* **Samples**: One sequence is one sample. A batch is comprised of one or more samples.
* **Time Steps**: One time step is one point of observation in the sample One sample is comprised of multiple time steps.
* **Features**: One feature is one observation at a time step. One time step is comprised of one or more features

```python
# lstm without an input layer
...
model = Sequential()
model.add(LSTM(32, input_shape=(3, 1)))
model.add(Dense(1))

This example maps onto our univariate time seris from the previous section that we split inot having 3 inpout time steps and 1 feature. We may have loaded our time series dataset from CSV or transformed it to a supervised learning problem in memory. It will have a two dimensional shape with some number of samples, 3 time steps per sample and 1 feature per time step, or [?, 3, 1]. We can do this by using the `reshape()` Numpy function. For example, if we have 7 samples and 3 time steps per sample for the input element of our time series, we can reshape it into [7,3,1] by providing a tuple to the `reshape()` function specifying the desired new shape of (7, 3, 1)

In [10]:
# transform input from [samples, features] to [samples, timesteps, features]
X = X.reshape((X.shape[0]), X.shape[1], 1)
print(X.shape)

(7, 3, 1)


### 6.4 Data Preparation Example

COnsider that you are in the current situation:

*I have two columns in my data file with 5k rows, column 1 is time (with 1 hour interval) and column 2 is the number of sales and I am trying to forecast the number of sales for future time steps. Help me to set the number of samples, time steps and features in this data for an LSTM?*

There are a few problems here:
* **Data Shape**: LSTMs expect 3D input, and it can be challenging to get your head around this the first time.
* **Sequence Lenght**: LSTMs don't like sequences of more than 200-400 time steps, so the data will need to be split into subsamples.

We will work through this example, broken down into the following 4 steps:

1. Load the Data
2. Drop the Time Column
3. Split Into Samples
4. Reshape Subsequences

### 6.4.1 Load the Data

In [73]:
# example of defining a dataset
from numpy import array

# define the dataset
data = list()
n = 5000
for i in range(n):
    data.append([i+1, (i+1)*10])
data = array(data)

In [14]:
data[:5, :]

array([[ 1, 10],
       [ 2, 20],
       [ 3, 30],
       [ 4, 40],
       [ 5, 50]])

In [16]:
data[:5]

array([[ 1, 10],
       [ 2, 20],
       [ 3, 30],
       [ 4, 40],
       [ 5, 50]])

In [70]:
print(data.shape)

(5000, 2)


### una pregunta salvaje se asoma a mi mente'

In [18]:
import numpy as np

In [67]:
# shape = (time_steps, features, channels)
arr = np.random.rand(2, 2, 3)



In [68]:
arr

array([[[0.69884517, 0.77929053, 0.78883682],
        [0.44210894, 0.09843502, 0.7784424 ]],

       [[0.25555054, 0.66120523, 0.12715955],
        [0.22733383, 0.02373867, 0.41410954]]])

In [69]:
slice1 = arr[:, :1,2]
slice1

array([[0.78883682],
       [0.12715955]])

In [49]:
import numpy as np

# A 2×2 “image” with RGB values
# Pixel (0,0) = [R=10, G=20, B=30]
# Pixel (0,1) = [R=40, G=50, B=60]
# Pixel (1,0) = [R=70, G=80, B=90]
# Pixel (1,1) = [R=100,G=110,B=120]
img = np.array([
    [[ 10,  20,  30],
     [ 40,  50,  60]],

    [[ 70,  80,  90],
     [100, 110, 120]]
])
print("img.shape =", img.shape)


img.shape = (2, 2, 3)


In [50]:
img

array([[[ 10,  20,  30],
        [ 40,  50,  60]],

       [[ 70,  80,  90],
        [100, 110, 120]]])

In [62]:
greens = img[:, :, 1]
print("greens:\n", greens)


greens:
 [[ 20  50]
 [ 80 110]]


In [52]:
top_row_green = img[0, :, 1]
print("top_row_green:", top_row_green)


top_row_green: [20 50]


### 6.4.2 Drop the Time Column

If your time series data is uniform over time and there is no missing values, we can drop the time column. If not, you may want to look at imputting the missing values, resampling the data to a new time scale, or developing a model that can handel missing values.
Here, we just drop the first column:

In [ ]:
# drop time
data = data[:,1]
print(data.shape)

(5000,)


### 6.4.3 Split Into Samples
LSTMs need to process samples where each sample is a single sequence of observations. In this case, 5000 time steps is too long; ; LSTMs work better with 200- to-400 time steps.
There are many ways to do this. For example, perhaps you need overlapping sequences, perhaps non-verlapping is good but your model needs state accros the sub-sequences and so on. In this example, we will split the 5k points into 25 sub-sequences. We will use the pata way.

In [75]:
# split into samples (e.g. 5000/200 = 25)
samples = list()
length = 200
# step over the 5k in jumps of 200
for i in range(0, n, length):
    # grab from i to i + 200
    sample = data[i:i + length]
    samples.append(sample)
print(len(samples))

25


### 6.4.4 Reshape Subsequences
The LSTM needs data with the format of [samples, timesteps, features]. We have 25 samples, 200 time steps per sample, and 1 feature. First we need to convert our list of arrays into a 2D NumPy array with the sahpe [25,200]

In [78]:
data = array(samples)
print(data.shape)

(25, 200)


Next we can use the reshape() fucntion to add one additional dimension for our single feature and use the existing columns as time steps instead.

In [79]:
# reshape into [samples, timesteps, features]
data = data.reshape((len(samples), length, 1))
print(data.shape)

(25, 200, 1)


## Chapter 7:
## How to Develop MLPs for Time Series Forecasting

After completing this tutorial, you will know:

* How to develop MLP models for univariate time series forecasting
* How to develop MLP models for multivariate time series forecasting
* How to develop MLP models for multi-step time series forecasting

### 7.1 Tutorial Overview

We will explore how to develop a suite of MLP models for tsf. 
The tutorial is divided into four parts; they are:

1. Univariate MLP Models
2. Multivariate MLP Models
3. Multi-step MLP Models
4. Multivsariate Multi-step MLP Models

### 7.2 Univariate MLP Models

### 7.2.1 Data Preparation

In [86]:
# univariate data preparation
from numpy import array

# split a univariate sequence into samples
def split_sequence(sequence, n_steps):
    X, y = list(), list()
    for i in range(len(sequence)):
      # find the end of this pattern
      end_ix = i + n_steps
      # check if we are beyond the sequence
      if end_ix > len(sequence)-1:
         break
      # gather input and outpu parts of the pattern
      seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
      X.append(seq_x)
      y.append(seq_y)

    return array(X), array(y)

# define an input sequence
raw_seq = array([i for i in range(10, 100, 10)])
# choose a number of time steps
n_steps = 3
# split into samples
# transform to a supervised learning problem
X, y = split_sequence(raw_seq, n_steps)
# show each example
for i in range(len(X)):
   print(X[i], y[i])

[10 20 30] 40
[20 30 40] 50
[30 40 50] 60
[40 50 60] 70
[50 60 70] 80
[60 70 80] 90


### 7.2.2 MLP Model
A simple MLP Model has a single hidden layer of nodes, and an output layer used to make a prediction. We can define it as follows:

```python
#define model
model = Sequential()
model.add(Dense(100, activation='relu', input_dim=n_steps))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

Important in the definition is the shape of the input; that is what the model expects as input for each sample in terms of the number of time steps.
Technically, the model will view each time step as a separate feature instead of separate time steps.
We almost always have multiple samples, therefore, the model will expect the input component of training data to have the dimensions or shape: [samples, features].

Once the model is defined, we can fit it on the training dataset.

```python
# fit model
model.fit(X, y, epochs = 2000, verbose=0)

After the model is fit, we can use it to make a prediction. We can predict the next value in the sequence by providing the input: [70,80,90]. And expecting the model to predict something like: [100].
The model expects the input shape to be two-dimensional with [samples, features], therefore, we must reshape the single input before making the prediction.

```python
# demonstrate prediction
x_input = array([70,80,90])
x_input = x_input.reshape((1, n_steps))
yhat = model.predict(x_input, verbose=0)

tying this together

In [3]:
import tensorflow as tf

In [4]:
from numpy import array
from keras.models import Sequential
from keras.layers import Dense

In [6]:
# univariate data preparation

# split a univariate sequence into samples
def split_sequence(sequence, n_steps):
    X, y = list(), list()
    for i in range(len(sequence)):
      # find the end of this pattern
      end_ix = i + n_steps
      # check if we are beyond the sequence
      if end_ix > len(sequence)-1:
         break
      # gather input and outpu parts of the pattern
      seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
      X.append(seq_x)
      y.append(seq_y)

    return array(X), array(y)

# define an input sequence
raw_seq = array([i for i in range(10, 100, 10)])
# choose a number of time steps
n_steps = 3
# split into samples
# transform to a supervised learning problem
X, y = split_sequence(raw_seq, n_steps)
#define model
model = Sequential()
model.add(Dense(100, activation='relu', input_dim=n_steps))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

/home/think/anaconda3/lib/python3.12/site-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2025-06-27 23:08:33.391662: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


In [8]:
# fit model
model.fit(X, y, epochs = 2000, verbose=1)

Epoch 1/2000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step - loss: 17.8720
Epoch 2/2000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step - loss: 17.8476
Epoch 3/2000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step - loss: 17.8231
Epoch 4/2000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step - loss: 17.7987
Epoch 5/2000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - loss: 17.7742
Epoch 6/2000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step - loss: 17.7497
Epoch 7/2000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step - loss: 17.7252
Epoch 8/2000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step - loss: 17.7007
Epoch 9/2000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step - loss: 17.6762
Epoch 10/2000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - loss: 17.6517
Epoch 11/2000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - loss: 17.6272
Epoch 12/2000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - loss: 17.6026
Epoch 13/2000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - loss: 17.5781
Epoch 14/2000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step - loss: 17.5535
Epoch 15/2000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 119ms/step - lo

In [9]:
# demonstrate prediction
x_input = array([70,80,90])
x_input = x_input.reshape((1, n_steps))
yhat = model.predict(x_input, verbose=0)

In [10]:
print(yhat)

[[100.01286]]
